In [1]:
import json
vars = json.load(open('v1.2.2_vars.json'))['Compound Name']

In [2]:
vars.keys(), list(vars.values())[0]

(dict_keys(['3hr.clt', '3hr.evspsblsoi', '3hr.evspsblveg', '3hr.ficeberg', '3hr.flandice', '3hr.friver', '3hr.hfds', '3hr.hfdsl', '3hr.hfls', '3hr.hfrunoffds', '3hr.hfss', '3hr.huss', '3hr.mpw', '3hr.mrro', '3hr.mrros', '3hr.mrso100', '3hr.mrsos', '3hr.pr', '3hr.prsn', '3hr.ps', '3hr.rlds', '3hr.rlus', '3hr.rsds', '3hr.rsntds', '3hr.rsus', '3hr.sfdsi', '3hr.srfrad', '3hr.swh', '3hr.tas', '3hr.tauuo', '3hr.tauvo', '3hr.tos', '3hr.tran', '3hr.tslsi', '3hr.ua100m', '3hr.va100m', '3hr.wdir', '3hr.wfo', '3hr.wpdir', '3hr.wpp', '3hrPt.bldep', '3hrPt.mpw', '3hrPt.swh', '3hrPt.uas', '3hrPt.vas', '3hrPt.wdir', '3hrPt.wpdir', '3hrPt.wpp', '6hr.mpw', '6hr.mpwswell', '6hr.mpwwindsea', '6hr.sduo', '6hr.sdvo', '6hr.swh', '6hr.swhmax', '6hr.swhswell', '6hr.swhwindsea', '6hr.wdir', '6hr.wdirswell', '6hr.wdirwindsea', '6hr.wpdir', '6hr.wpdirswell', '6hr.wpdirwindsea', '6hr.wpp', '6hr.wppswell', '6hr.wppwindsea', '6hrLev.bs550aer', '6hrLev.ec550aer', '6hrLev.hus', '6hrLev.ps', '6hrLev.ta', '6hrLev.ua', 

In [3]:
def common_key_val(data):
    return {
        k: v for k, v in data[0].items()
        if all(d.get(k) == v for d in data[1:])
    }

In [4]:
def common_by_var(var):
    root = {}
    for i in vars:
        if vars[i]['variableRootDD'] in root:
            root[vars[i]['variableRootDD']].append(vars[i])
        else:
            root[vars[i]['variableRootDD']] = [vars[i]]
    
    multiples = [i for i in root if len(root[i]) > 1 ]
    
    keys = []


    for i in multiples:
        data = root[i]
        common = common_key_val(data)
        # keys.extend(list(common.keys()))
        keys.append(common)
        
    common_keys = set.intersection(*(set(d.keys()) for d in keys))
    
    return common_keys

        
            

In [5]:
# common_by_var('standard_name')

In [6]:
keygroups = {}
for k in list(vars.values())[0].keys():
    common = common_by_var(k)
    # print(f"Common keys for {k}: {common}")
    keygroups[k] = common
    
    

In [7]:
from collections import defaultdict
# Reverse the mapping
grouped = defaultdict(set)

for key, value_set in keygroups.items():
    grouped[frozenset(value_set)].add(key)
    
grouped = list(grouped)

In [8]:
varrtdd = {}
for k, v in vars.items():
    if v['variableRootDD'] not in varrtdd:
        varrtdd[v['variableRootDD']] = {k: v[k] for k in {'standard_name', 'type', 'variableRootDD'} if k in v}
    else:
        assert varrtdd[v['variableRootDD']] == {k: v[k] for k in {'standard_name', 'type', 'variableRootDD'} if k in v}

In [24]:
# write jsonld 
folder = '../src-data/variable-root/'


for s in varrtdd.values():
    name  = s['variableRootDD'].lower().replace(' ', '-').replace('_', '-')
    
    jld = {
        'id': name,
        'validation-key': name,
        'ui-label'  : s['variableRootDD'],
        'data-type' : s['type'],
        'standard-name': s['standard_name'],
        'type': ['wcrp:variable-root','variable-registry',],
        '@context': '_context_'
    }
    
    with open(f"{folder}{name}.json", 'w') as f:
        json.dump(jld, f, indent=4, ensure_ascii=False)

        
ctx = {
    "@context": {
        "@base": "https://wcrp-cmip.github.io/Variable-Registry/variable-root/",
        "@vocab": "https://wcrp-cmip.github.io/Variable-Registry/variable-root/",
        "standard-name": {
                "@type": "@id",
                "@context": "https://wcrp-cmip.github.io/CF/standard-name/_context_"
            }
    },
    "@embed": "@always"
}

with open(f'{folder}_context_', 'w') as f:
    json.dump(ctx, f, indent=4, ensure_ascii=False)

clear!

In [10]:
import glob
cm = glob.glob('../src-data/cell-method/*.json')
cellmethods = {}
for i in cm: 
    with open(i, 'r') as f:
        c = json.load(f)
        cellmethods[c['description']] = c['id']
        
len(cellmethods)

80

In [25]:

import re 
from collections import OrderedDict
varlist = []
dims = []
for vr in vars.values():
    
    # del vr['type'], vr['standard_name']
    vr = {re.sub(r'(?<!^)(?=[A-Z])', '_', k).lower().replace('_', '-'): v for k, v in vr.items()}

    vr['cell-methods']  = cellmethods.get(vr['cell-methods'], f"Error: {vr['cell-methods']} not found in cell methods, please register")
    vr['dimensions'] = vr['dimensions'].lower().split(' ')
    vr['model-realm'] = vr['modeling-realm'].lower().replace(' ', '-').replace('_', '-')
    
    dims.extend(vr['dimensions'])
    
    # <temporalLabelDD>- <verticalLabelDD>-<horizontalLabelDD>-<areaLabelDD>
    # print(vr.keys())
    
    try:
        branding = vr['branded-variable-name'].split('_')[1].split('-') 
    except IndexError:
        print(f"Error: {vr['branded-variable-name']} does not have the expected format")
        branding  = [vr['branded-variable-name']]*5

    dummy = OrderedDict({
        "id": vr['branded-variable-name'],
        "validation-key": vr['branded-variable-name'],
        "ui-label": vr['long-name'],
        "description": vr['comment'],
        "variable-root":vr['variable-root-d-d'],
        "temporal-label": branding[0],
        "vertical-label": branding[1],
        "horizontal-label": branding[2],
        "area-label": branding[3],
        "remove-cmip6-table": vr['cmip6-table'],
    })
    
    del vr['long-name'], vr['comment'], vr['branded-variable-name'], vr['variable-root-d-d'], vr['uid'],vr['cmip6-table'],vr['modeling-realm']
    
    dummy.update(vr)
 
    dummy['@context'] = '_context_'
    dummy['type'] = ['wcrp:variable', 'variable-registry']

    with open(f"../src-data/variable/{dummy['id']}.json", 'w') as f:
        json.dump(dummy, f, indent=4, ensure_ascii=False) 
    
    varlist.append(dummy)
        
ctx = {
"@context": {
    "@base": "https://wcrp-cmip.github.io/Variable-Registry/variable/",
    "@vocab": "https://wcrp-cmip.github.io/Variable-Registry/variable/",
    "cell-methods": {
            "@type": "@id",
            "@context": "https://wcrp-cmip.github.io/Variable-Registry/cell-method/_context_"
        },
    "temporal-label": {
            "@type": "@id",
            "@context": "https://wcrp-cmip.github.io/Variable-Registry/temporal-label/_context_"
        },
    "vertical-label": {
            "@type": "@id",
            "@context": "https://wcrp-cmip.github.io/Variable-Registry/vertical-label/_context_"
        },
    "horizontal-label": {
            "@type": "@id",
            "@context": "https://wcrp-cmip.github.io/Variable-Registry/horizontal-label/_context_"
        },
    "area-label": {
            "@type": "@id",
            "@context": "https://wcrp-cmip.github.io/Variable-Registry/area-label/_context_"
        },
    "dimensions": {
            "@type": "@id",
            "@context": "https://wcrp-cmip.github.io/Variable-Registry/area-label/_context_"
        },
    "model-realm": {
            "@type": "@id",
            "@context": "https://wcrp-cmip.github.io/WCRP-universe/realm/_context_"
        },
    
},
"@embed": "@always"
}

with open(f'../src-data/variable/_context_', 'w') as f:
    json.dump(ctx, f, indent=4, ensure_ascii=False)
dummy


Error: unknown01 does not have the expected format
Error: unknown02 does not have the expected format
Error: unknown03 does not have the expected format
Error: unknown04 does not have the expected format
Error: unknown05 does not have the expected format


OrderedDict([('id', 'siv_tavg-u-hxy-si'),
             ('validation-key', 'siv_tavg-u-hxy-si'),
             ('ui-label', 'Y-Component of Sea-Ice Velocity'),
             ('description',
              'Y-component of sea-ice velocity on native model grid.'),
             ('variable-root', 'siv'),
             ('temporal-label', 'tavg'),
             ('vertical-label', 'u'),
             ('horizontal-label', 'hxy'),
             ('area-label', 'si'),
             ('remove-cmip6-table', 'SImon'),
             ('frequency', 'mon'),
             ('standard-name', 'sea_ice_y_velocity'),
             ('units', 'm s-1'),
             ('cell-methods',
              'Error: area: time: mean where sea_ice (mask=siconc) not found in cell methods, please register'),
             ('cell-measures', '::MODEL'),
             ('dimensions', ['longitude', 'latitude', 'time']),
             ('out-name', 'siv'),
             ('type', ['wcrp:variable', 'variable-registry']),
             ('positive', ''),


```Dimension names (see Appendix C)
Temporal sampling options (see Appendix F)
Horizontal dimensional structure options (see Appendix F)
Vertical dimension options (see Appendix F)
Area options (see Appendix F)


<variableRootDD>_<temporalLabelDD>- <verticalLabelDD>-<horizontalLabelDD>-<areaLabelDD>.

In [ ]:
tables = {}

for v in varlist:
    table = v['remove-cmip6-table']
    if table not in tables:
        tables[table] = []
    tables[table].append(v['id'])
    
tables.keys()

for t in tables:
    tid=t.lower().replace(' ', '-').replace('_', '-')
    tv = {
        "id":tid,
        "validation-key": tid,
        "ui-label": t,
        "description": f"CMIP6 CMOR table {t} containing variables",
        "variables":tables[t],
        "@context": "_context_",
        "type": ["wcrp:mip-table", "variable-registry","cmip6"]
        }
    
    with open(f"../src-data/cmip6-mip-table/{tid}.json", 'w') as f:
        json.dump(tv, f, indent=4, ensure_ascii=False)
    
    
    
ctx = {
"@context": {
    "@base": "https://wcrp-cmip.github.io/Variable-Registry/cmip6-mip-table/",
    "@vocab": "https://wcrp-cmip.github.io/Variable-Registry/cmip6-mip-table/",
    "variables": {
            "@type": "@id",
            "@context": "https://wcrp-cmip.github.io/Variable-Registry/variable/_context_"
        },
},
"@embed": "@always"
}

with open(f'../src-data/cmip6-mip-table/_context_', 'w') as f:
    json.dump(ctx, f, indent=4, ensure_ascii=False)
dummy

FileNotFoundError: [Errno 2] No such file or directory: '../src-data/cmip6-tables/3hr.json'